## Performance Measurement and Improvement Techniques

### Goal
>In image processing, since you are dealing with large number of operations per second, it is mandatory that your code is not only providing the correct solution, but also in the fastest manner. So in this chapter, you will learn
To measure the performance of your code.
Some tips to improve the performance of your code.
You will see these functions : cv2.getTickCount, cv2.getTickFrequency etc.
Apart from OpenCV, Python also provides a module time which is helpful in measuring the time of execution. Another module profile helps to get detailed report on the code, like how much time each function in the code took, how many times the function was called etc. But, if you are using IPython, all these features are integrated in an user-friendly manner. We will see some important ones, and for more details, check links in Additional Resouces section.

In [1]:
import cv2
import numpy as np

e1 = cv2.getTickCount()
# your code execution
e2 = cv2.getTickCount()
time = (e2 - e1)/ cv2.getTickFrequency()

print(time)

img1 = cv2.imread('messi5.png')

e1 = cv2.getTickCount()
for i in range(5,49,2):
    img1 = cv2.medianBlur(img1,i)
e2 = cv2.getTickCount()
t = (e2 - e1)/cv2.getTickFrequency()
print(t)

cv2.useOptimized()
cv2.setUseOptimized(True)

%timeit res = cv2.medianBlur(img1, 49)
x = 5
z = np.uint8([5])
%timeit y=x**2
%timeit y=x*x
%timeit y=z*z
%timeit y=np.square(z)

print('aaaaaaaaaa')
cv2.setUseOptimized(False)
z = np.uint8([5])
cv2.useOptimized()
%timeit res = cv2.medianBlur(img1,49)
%timeit y=x**2
%timeit y=x*x
%timeit y=z*z
%timeit y=np.square(z)


4.5905e-05
1.255706988
74.5 ms ± 27.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
723 ns ± 108 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
110 ns ± 1.58 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
1.07 µs ± 7.53 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.13 µs ± 19 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
aaaaaaaaaa
55 ms ± 1.07 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
683 ns ± 157 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
109 ns ± 2.88 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
1.36 µs ± 250 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.26 µs ± 208 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [8]:
%timeit res = cv2.medianBlue(img1, 49)

AttributeError: module 'cv2' has no attribute 'medianBlue'